In [10]:
from py2neo import Graph

### My parameters
#graph = Graph("http://neo4j.lis.ic.unicamp.br:80/db/data/", )
graph = Graph(host = "neo4j.lis.ic.unicamp.br", http_port = 80, bolt = False, secure = False)


if(graph):
    print ("Connected")
    print (graph)

graph.begin(autocommit = True)

Connected
<Graph uri='http://neo4j.lis.ic.unicamp.br:80/db/data/'>


In [17]:
algorithm = "NormalizedLevenshtein"# NormalizedLevenshtein, JaroWinkler, Cosine, Jaccard, MetricLCS, OptimalStringAlignment
ontology  = 'hfo'

In [18]:
import requests
import json  

def findEntities(text, n):
    payload = { 'text': text, 'n': n, 'ontology':ontology, 'algorithm':algorithm } 
#    r = requests.post('http://ontomatch.lis.ic.unicamp.br/api/rest/resources', data=json.dumps(payload))
    r = requests.post('http://10.1.1.198:8080/ontomatch/rest/resources', data=json.dumps(payload))
    
    
#     restReturn = r.text.replace('callback(', '')
#     restReturn = restReturn.replace('})', '}')
    returnJson = json.loads(r.text)  
    return returnJson


json.dumps(findEntities("Left ventricular hypertrophy on admission electrocardiogram", 3), indent=3, sort_keys=True)

'[\n   {\n      "label": "left ventricular hypertrophy on ecg",\n      "matchedWords": [],\n      "similarity": 0.5932203389830508,\n      "uri": "http://bmi.utah.edu/ontologies/hfontology/C0232306"\n   },\n   {\n      "label": "Left ventricular ejection fraction decreased",\n      "matchedWords": [],\n      "similarity": 0.5084745762711864,\n      "uri": "http://bmi.utah.edu/ontologies/hfontology/C1096403"\n   },\n   {\n      "label": "Left Ventricular Hypertrophy",\n      "matchedWords": [],\n      "similarity": 0.47457627118644063,\n      "uri": "http://bmi.utah.edu/ontologies/hfontology/C0149721"\n   }\n]'

# Clinical Features Similarities

In [16]:
from py2neo import Node, Relationship, NodeSelector
selector = NodeSelector(graph)

floorLimiar   = 0.35
ceilingLimiar = 1
result = selector.select("ClinicalFeature")
nodelist=list(result)

# print(nodelist)

countFeatures=0
countHits=0

with open('out.txt', 'w') as f:

    for node in nodelist:
        countFeatures += 1
        print("---------------\n\nClinical Feature: {0}".format(node['label']), file=f)
        similarEntities = findEntities(node['label'], 10)
        for entity in similarEntities:
            if(entity['similarity'] >= floorLimiar and entity['similarity'] <= ceilingLimiar):
                #print("{0}".format(entity['label']), file=f)
                print("{1} (similarity of {0}) [uri {2}]".format(entity['similarity'], entity['label'], entity['uri'] ), file=f)
                countHits += 1
        print(file=f)

    
f.close() 

print("CountFeatures: {0}".format(countFeatures))
print("CountHits    : {0}".format(countHits))
print("%            : {0}".format((countHits/countFeatures)*100))


CountFeatures: 39
CountHits    : 365
%            : 935.8974358974359


## Generating CSV for plotting

In [ ]:
from py2neo import Node, Relationship, NodeSelector
import numpy as np

selector = NodeSelector(graph)

result = selector.select("ClinicalFeature")
nodelist=list(result)

ceilingLimiar = 1

intervalResult = []
compiledResult = []

# intervals = [0.9, 0.95]
# intervals = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
intervals = [0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

#algorithms = ["Cosine", "NormalizedLevenshtein", "Jaccard", "JaroWinkler"]

for algorithm in algorithms:
    for floorLimiar in intervals:
        print(str(algorithm)+"_"+str(floorLimiar))
        countFeatures=0
        countHits=0
        intervalResult = []
        intervalResult.append(str(algorithm)+"_"+str(floorLimiar))

        for node in nodelist:
            countFeatures += 1
            countHits = 0

            similarEntities = findEntities(node['label'], 100)

            for entity in similarEntities:
                if(entity['similarity'] >= floorLimiar and entity['similarity'] <= ceilingLimiar):
                    countHits += 1

            intervalResult.append(countHits)
            
            
        npIntervalResult = np.array(intervalResult[2:]).astype(np.float) #https://docs.scipy.org/doc/numpy/reference/routines.statistics.html
        intervalResult.append(np.amin(npIntervalResult)) #min max average median std var
        intervalResult.append(np.amax(npIntervalResult))
        intervalResult.append(np.average(npIntervalResult))
        intervalResult.append(np.median(npIntervalResult))
        intervalResult.append(np.std(npIntervalResult))
        intervalResult.append(np.var(npIntervalResult))
        
        compiledResult.append(intervalResult)

    
    
import csv
with open('output/compilation.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(compiledResult)
f.close()